# Data Wrangling

## Spark Loading

In [1]:
%load_ext sparkmagic.magics

In [2]:
import pickle
path_data = "../data/"

In [3]:
# pull from lfs
! git lfs pull

In [4]:
import os
from IPython import get_ipython
username = os.environ['RENKU_USERNAME']
server = "http://iccluster029.iccluster.epfl.ch:8998"

# set the application name as "<your_gaspar_id>-homework3"
get_ipython().run_cell_magic(
    'spark',
    line='config', 
    cell="""{{ "name": "{0}-homework3", "executorMemory": "4G", "executorCores": 4, "numExecutors": 10, "driverMemory": "4G"}}""".format(username)
)

In [5]:
get_ipython().run_line_magic(
    "spark", "add -s {0}-homework3 -l python -u {1} -k".format(username, server)
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5946,application_1652960972356_1257,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [6]:
%%spark
print('We are using Spark %s' % spark.version)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We are using Spark 2.3.2.3.1.4.0-315

## Stops 15km from Zürich HB (8503000) (lat, lon) = (47.378177, 8.540192)

In [7]:
%%spark
all_stops = spark.read.csv('/data/sbb/csv/allstops/stop_locations.csv')
all_stops.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+----+----+
|    _c0|                 _c1|             _c2|             _c3| _c4| _c5|
+-------+--------------------+----------------+----------------+----+----+
|1100006|Zell (Wiesental),...|47.7046317515335|7.84777215468897|null|null|
|1100008|Zell (Wiesental),...|47.7100842702352|7.85964788274668|null|null|
|1100009|Zell (Wiesental),...|47.7131911044794|7.86290876722849|null|null|
|1100010|           Atzenbach|47.7146175266411| 7.8723500608659|null|null|
|1100011|     Mambach, Brücke|47.7282088873189| 7.8774704579861|null|null|
|1100012|  Mambach, Mühlschau|47.7340818684375| 7.8813871126254|null|null|
|1100013|  Mambach, Silbersau|47.7395192233867|7.88223152899259|null|null|
|1100014|Fröhnd (Schwarzw)...|47.7543663509316|7.88913059037559|null|null|
|1100015|Fröhnd (Schwarzw)...|47.7605926689054|7.88553732923861|null|null|
|1100016|     Wembach (Baden)|47.7728317637339|7.88772023537933|null|null|
+-------+----------------

In [8]:
%%spark
all_stops = all_stops.withColumnRenamed('_c0','stop_id')
all_stops = all_stops.withColumnRenamed('_c1','stop_name')
all_stops = all_stops.withColumnRenamed('_c2','lat')
all_stops = all_stops.withColumnRenamed('_c3','lon')
all_stops = all_stops.drop('_c4','_c5')
all_stops.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+
|stop_id|           stop_name|             lat|             lon|
+-------+--------------------+----------------+----------------+
|1100006|Zell (Wiesental),...|47.7046317515335|7.84777215468897|
|1100008|Zell (Wiesental),...|47.7100842702352|7.85964788274668|
|1100009|Zell (Wiesental),...|47.7131911044794|7.86290876722849|
|1100010|           Atzenbach|47.7146175266411| 7.8723500608659|
|1100011|     Mambach, Brücke|47.7282088873189| 7.8774704579861|
|1100012|  Mambach, Mühlschau|47.7340818684375| 7.8813871126254|
|1100013|  Mambach, Silbersau|47.7395192233867|7.88223152899259|
|1100014|Fröhnd (Schwarzw)...|47.7543663509316|7.88913059037559|
|1100015|Fröhnd (Schwarzw)...|47.7605926689054|7.88553732923861|
|1100016|     Wembach (Baden)|47.7728317637339|7.88772023537933|
+-------+--------------------+----------------+----------------+
only showing top 10 rows

In [9]:
%%spark
import pyspark.sql.functions as F
zur_hp_lat = 47.378177
zur_hp_lon = 8.540192

all_stops_distance = all_stops.withColumn("a", (
        F.pow(F.sin(F.radians(zur_hp_lat - F.col("lat")) / 2), 2) +
        F.cos(F.radians(F.col("lat"))) * F.cos(F.radians(F.lit(zur_hp_lat))) *
        F.pow(F.sin(F.radians(zur_hp_lon - F.col("lon")) / 2), 2)
    )).withColumn("distance", F.atan2(F.sqrt(F.col("a")), F.sqrt(-F.col("a") + 1)) * 12742000)

all_stops_distance.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+--------------------+------------------+
|stop_id|           stop_name|             lat|             lon|                   a|          distance|
+-------+--------------------+----------------+----------------+--------------------+------------------+
|1100006|Zell (Wiesental),...|47.7046317515335|7.84777215468897|2.475398503604094...|  63396.0142787592|
|1100008|Zell (Wiesental),...|47.7100842702352|7.85964788274668|2.445985737384344...| 63018.24892966985|
|1100009|Zell (Wiesental),...|47.7131911044794|7.86290876722849|2.446306465224431...| 63022.38043559904|
|1100010|           Atzenbach|47.7146175266411| 7.8723500608659|2.409494241610368E-5|62546.396144882805|
|1100011|     Mambach, Brücke|47.7282088873189| 7.8774704579861|2.456510835903595E-5| 63153.68854022285|
|1100012|  Mambach, Mühlschau|47.7340818684375| 7.8813871126254|2.469960438154199...| 63326.33982721011|
|1100013|  Mambach, Silbersau|47.7395192233867|7.882231

In [10]:
%%spark
close_stops = all_stops_distance.filter(all_stops_distance['distance'] <= 15000).drop('a','distance')
close_stops.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------------+----------------+----------------+
|      stop_id|           stop_name|             lat|             lon|
+-------------+--------------------+----------------+----------------+
|          176|Zimmerberg-Basist...|47.3516780901371|8.52195777551452|
|      8500926|Oetwil a.d.L., Sc...|47.4236270123012| 8.4031825286317|
|      8502075|Zürich Flughafen,...|47.4510244676285|8.56372943623189|
|      8502186|Dietikon Stoffelbach|47.3933267759652|8.39896044679575|
|    8502186:0|Dietikon Stoffelbach|47.3933997509195|8.39894248049007|
|8502186:0:1/2|Dietikon Stoffelbach|47.3933997509195|8.39894248049007|
|     8502186P|Dietikon Stoffelbach|47.3933997509195|8.39894248049007|
|      8502187|Rudolfstetten Hof...|47.3646702178563|8.37695172233176|
|    8502187:0|Rudolfstetten Hof...|47.3647371479356|8.37703257070734|
|8502187:0:1/2|Rudolfstetten Hof...|47.3647371479356|8.37703257070734|
+-------------+--------------------+----------------+----------------+
only s

In [11]:
%%spark
close_stops.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2122

## Weekly timetable

Considered period: May 08-14, 2019 (then we exclude the weekends)

In [12]:
%%spark
all_trips1 = spark.read.csv('/data/sbb/csv/stop_times/2019/05/08/stop_times.txt', header=True)
#all_trips2 = spark.read.csv('/data/sbb/csv/stop_times/2019/05/15/stop_times.txt', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
%%spark
all_trips1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10862563

In [14]:
%%spark
#all_trips2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
%%spark
all_trips1.filter(all_trips1.arrival_time==all_trips1.departure_time).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10413974

In [16]:
%%spark
#all_trips_tot = all_trips1.union(all_trips2)
all_trips_tot = all_trips1
all_trips_tot.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------+--------------+-------+-------------+-----------+-------------+
|            trip_id|arrival_time|departure_time|stop_id|stop_sequence|pickup_type|drop_off_type|
+-------------------+------------+--------------+-------+-------------+-----------+-------------+
|1.TA.1-84-j19-1.1.H|    06:13:00|      06:13:00|8572249|            1|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:14:00|      06:14:00|8577508|            2|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:15:00|      06:15:00|8581070|            3|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:16:00|      06:16:00|8578360|            4|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:17:00|      06:17:00|8583448|            5|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:18:00|      06:19:00|8578359|            6|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:24:00|      06:24:00|8578358|            7|          0|            0|
|1.TA.1-84-j19-1.1.H

In [17]:
%%spark
routes1 = spark.read.csv('/data/sbb/csv/trips/2019/05/08/trips.txt', header=True)
#routes2 = spark.read.csv('/data/sbb/csv/trips/2019/05/15/trips.txt', header=True)
#routes_tot = routes1.union(routes2)
routes_tot = routes1
routes_tot.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+--------------------+-------------------+---------------+------------+
|   route_id|service_id|             trip_id|      trip_headsign|trip_short_name|direction_id|
+-----------+----------+--------------------+-------------------+---------------+------------+
| 1-85-j19-1|  TA+b0001| 2.TA.1-85-j19-1.1.H|Schöftland, Bahnhof|          85003|           0|
|1-1-C-j19-1|  TA+b0001|5.TA.1-1-C-j19-1.3.R| Zofingen, Altachen|            108|           1|
|1-1-C-j19-1|  TA+b0001|7.TA.1-1-C-j19-1.3.R| Zofingen, Altachen|            112|           1|
|1-1-C-j19-1|  TA+b0001|9.TA.1-1-C-j19-1.3.R| Zofingen, Altachen|            116|           1|
|1-1-C-j19-1|  TA+b0001|11.TA.1-1-C-j19-1...| Zofingen, Altachen|            120|           1|
|1-1-C-j19-1|  TA+b0001|13.TA.1-1-C-j19-1...| Zofingen, Altachen|            124|           1|
|1-1-C-j19-1|  TA+b0001|15.TA.1-1-C-j19-1...| Zofingen, Altachen|            128|           1|
|1-1-C-j19-1|  TA+b0001|17.TA.1-1-C-j19-1...| Zofi

In [18]:
%%spark
trips_with_routes = all_trips_tot.join(routes_tot, all_trips_tot.trip_id == routes_tot.trip_id)
trips_with_routes.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+------------+-------------+-----------+-------------+------------+----------+--------------------+--------------+---------------+------------+
|             trip_id|arrival_time|departure_time|     stop_id|stop_sequence|pickup_type|drop_off_type|    route_id|service_id|             trip_id| trip_headsign|trip_short_name|direction_id|
+--------------------+------------+--------------+------------+-------------+-----------+-------------+------------+----------+--------------------+--------------+---------------+------------+
|1.TA.12-1-A-j19-1...|    01:30:00|      01:30:00|8505000:0:11|            1|          0|            0|12-1-A-j19-1|  TA+b099q|1.TA.12-1-A-j19-1...|        Sursee|          31100|           0|
|1.TA.12-1-A-j19-1...|    01:35:00|      01:35:00| 8502021:0:1|            2|          0|            0|12-1-A-j19-1|  TA+b099q|1.TA.12-1-A-j19-1...|        Sursee|          31100|           0|
|1.TA.12-1-A-j19-1...|    01:36:00|

In [19]:
%%spark
trips_with_routes = trips_with_routes.select(all_trips_tot.trip_id, 'service_id').distinct()
trips_with_routes.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+
|             trip_id|service_id|
+--------------------+----------+
|1.TA.12-1-A-j19-1...|  TA+b099q|
|1.TA.16-440-j19-1...|  TA+b0007|
|1.TA.23-792-j19-1...|  TA+b000o|
|1.TA.26-18-j19-1.1.H|  TA+b0b46|
|1.TA.26-833-j19-1...|  TA+b002d|
|1.TA.26-853-j19-1...|  TA+b001t|
|1.TA.30-32-Y-j19-...|  TA+b09sz|
| 1.TA.5-21-j19-1.1.H|  TA+b08w1|
|1.TA.6-M13-j19-1.1.H|  TA+b0bou|
|1.TA.61-211-Y-j19...|  TA+b0786|
+--------------------+----------+
only showing top 10 rows

In [20]:
%%spark
cal1 = spark.read.csv('/data/sbb/csv/calendar/2019/05/08/calendar.txt', header=True)
#cal2 = spark.read.csv('/data/sbb/csv/calendar/2019/05/15/calendar.txt', header=True)
#cal_tot = cal1.union(cal2).distinct()
cal_tot = cal1.distinct()
cal_tot.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+---------+--------+------+--------+------+----------+--------+
|service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|start_date|end_date|
+----------+------+-------+---------+--------+------+--------+------+----------+--------+
|  TA+b06hv|     0|      0|        0|       0|     0|       0|     1|  20181209|20191214|
|  TA+b0036|     0|      0|        0|       0|     0|       0|     1|  20181209|20191214|
|  TA+b0ha1|     0|      0|        0|       0|     0|       1|     0|  20181209|20191214|
|  TA+b0nha|     1|      1|        1|       1|     0|       0|     1|  20181209|20191214|
|  TA+b022q|     1|      1|        1|       1|     0|       0|     0|  20181209|20191214|
|  TA+b025m|     0|      0|        0|       0|     1|       0|     0|  20181209|20191214|
|  TA+b0ms4|     0|      0|        0|       0|     0|       0|     1|  20181209|20191214|
|  TA+b0nvl|     0|      0|        0|       0|     0|       1|     1|  20181209|20191214|
|  TA+b0nz

In [21]:
%%spark
cal_filtered = cal_tot.filter((cal_tot.monday == 1) | (cal_tot.tuesday == 1) | (cal_tot.wednesday == 1) | (cal_tot.thursday == 1) | (cal_tot.friday == 1))
cal_filtered_dates = cal_filtered.filter((cal_tot.start_date <= '20190513') & (cal_tot.end_date >= '20190518'))
cal_filtered_dates.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+---------+--------+------+--------+------+----------+--------+
|service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|start_date|end_date|
+----------+------+-------+---------+--------+------+--------+------+----------+--------+
|  TA+b0nha|     1|      1|        1|       1|     0|       0|     1|  20181209|20191214|
|  TA+b022q|     1|      1|        1|       1|     0|       0|     0|  20181209|20191214|
|  TA+b025m|     0|      0|        0|       0|     1|       0|     0|  20181209|20191214|
|  TA+b0nz0|     0|      0|        0|       0|     1|       1|     0|  20181209|20191214|
|  TA+b0o2i|     1|      0|        0|       1|     0|       0|     0|  20181209|20191214|
|  TA+b0obi|     1|      1|        1|       1|     1|       0|     0|  20181209|20191214|
|  TA+b0obp|     0|      1|        0|       0|     1|       1|     1|  20181209|20191214|
|  TA+b0gtx|     1|      1|        1|       1|     1|       0|     1|  20181209|20191214|
|  TA+b0g2

In [22]:
%%spark
service_ids = cal_filtered_dates.drop('start_date', 'end_date', 'saturday', 'sunday')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
%%spark
service_ids.select('service_id')
trips_in_period = trips_with_routes.join(service_ids, service_ids.service_id == trips_with_routes.service_id,'inner').drop(trips_with_routes.service_id).distinct()
trips_in_period.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+------+-------+---------+--------+------+
|             trip_id|service_id|monday|tuesday|wednesday|thursday|friday|
+--------------------+----------+------+-------+---------+--------+------+
|1.TA.12-1-A-j19-1...|  TA+b099q|     0|      1|        1|       0|     1|
|1.TA.16-440-j19-1...|  TA+b0007|     1|      1|        1|       1|     1|
|1.TA.23-792-j19-1...|  TA+b000o|     0|      0|        0|       0|     1|
|1.TA.26-18-j19-1.1.H|  TA+b0b46|     1|      1|        1|       1|     1|
|1.TA.26-833-j19-1...|  TA+b002d|     0|      0|        0|       0|     1|
|1.TA.30-32-Y-j19-...|  TA+b09sz|     0|      1|        0|       0|     0|
|1.TA.6-M13-j19-1.1.H|  TA+b0bou|     0|      0|        0|       0|     1|
|1.TA.61-211-Y-j19...|  TA+b0786|     1|      1|        1|       1|     1|
|1.TA.61-40-Y-j19-...|  TA+b0001|     1|      1|        1|       1|     1|
|1.TA.61-479-Y-j19...|  TA+b04d0|     0|      0|        1|       1|     1|
+--------------------+---

In [24]:
%%spark
final_trips = all_trips_tot.join(trips_in_period, trips_in_period.trip_id == all_trips_tot.trip_id).distinct().drop(all_trips_tot.trip_id)
final_trips.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+------------+-------------+-----------+-------------+--------------------+----------+------+-------+---------+--------+------+
|arrival_time|departure_time|     stop_id|stop_sequence|pickup_type|drop_off_type|             trip_id|service_id|monday|tuesday|wednesday|thursday|friday|
+------------+--------------+------------+-------------+-----------+-------------+--------------------+----------+------+-------+---------+--------+------+
|    01:30:00|      01:30:00|8505000:0:11|            1|          0|            0|1.TA.12-1-A-j19-1...|  TA+b099q|     0|      1|        1|       0|     1|
|    01:35:00|      01:35:00| 8502021:0:1|            2|          0|            0|1.TA.12-1-A-j19-1...|  TA+b099q|     0|      1|        1|       0|     1|
|    01:36:00|      01:37:00| 8502028:0:1|            3|          0|            0|1.TA.12-1-A-j19-1...|  TA+b099q|     0|      1|        1|       0|     1|
|    01:39:00|      01:39:00| 8502012:0:1|            4|        

In [25]:
%%spark
all_trips_tot.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10862563

In [26]:
%%spark
final_trips.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7008267

### Considering working hours from 8:00 to 19:59

In [27]:
%%spark
import pyspark.sql.functions as F

final_trips = final_trips.filter(F.hour(final_trips.arrival_time).between(8,19) & F.hour(final_trips.departure_time).between(8,19))
final_trips.show(10, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+-----------+-------------+-----------+-------------+--------------------+----------+------+-------+---------+--------+------+
|arrival_time|departure_time|stop_id    |stop_sequence|pickup_type|drop_off_type|trip_id             |service_id|monday|tuesday|wednesday|thursday|friday|
+------------+--------------+-----------+-------------+-----------+-------------+--------------------+----------+------+-------+---------+--------+------+
|10:41:00    |10:41:00      |8503064    |1            |0          |0            |1.TA.26-18-j19-1.1.H|TA+b0b46  |1     |1      |1        |1       |1     |
|10:45:00    |10:45:00      |8503065:0:1|2            |0          |0            |1.TA.26-18-j19-1.1.H|TA+b0b46  |1     |1      |1        |1       |1     |
|10:46:00    |10:46:00      |8503074    |3            |0          |0            |1.TA.26-18-j19-1.1.H|TA+b0b46  |1     |1      |1        |1       |1     |
|10:47:00    |10:47:00      |8503068    |4            |0          |0  

In [28]:
%%spark
final_trips.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4940426

### Keeping only trip_ids that contains a stop in the area of Zurich

In [29]:
%%spark
final_close_trips_ids = final_trips.join(close_stops, final_trips.stop_id == close_stops.stop_id, 'inner').select('trip_id').distinct()
final_close_trips_ids.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|             trip_id|
+--------------------+
|1.TA.26-18-j19-1.1.H|
|1.TA.63-138-Y-j19...|
|1001.TA.26-213-j1...|
|1014.TA.26-520-j1...|
|1017.TA.26-151-j1...|
|1023.TA.26-4-j19-...|
|1025.TA.26-212-j1...|
|1029.TA.26-LAF-j1...|
|103.TA.26-510-j19...|
|1039.TA.26-200-j1...|
+--------------------+
only showing top 10 rows

In [30]:
%%spark
final_close_trips = final_trips.join(final_close_trips_ids, final_trips.trip_id == final_close_trips_ids.trip_id).drop(final_close_trips_ids.trip_id).distinct()
final_close_trips.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+------------+-------------+-----------+-------------+----------+------+-------+---------+--------+------+--------------------+
|arrival_time|departure_time|     stop_id|stop_sequence|pickup_type|drop_off_type|service_id|monday|tuesday|wednesday|thursday|friday|             trip_id|
+------------+--------------+------------+-------------+-----------+-------------+----------+------+-------+---------+--------+------+--------------------+
|    10:41:00|      10:41:00|     8503064|            1|          0|            0|  TA+b0b46|     1|      1|        1|       1|     1|1.TA.26-18-j19-1.1.H|
|    10:45:00|      10:45:00| 8503065:0:1|            2|          0|            0|  TA+b0b46|     1|      1|        1|       1|     1|1.TA.26-18-j19-1.1.H|
|    10:46:00|      10:46:00|     8503074|            3|          0|            0|  TA+b0b46|     1|      1|        1|       1|     1|1.TA.26-18-j19-1.1.H|
|    10:47:00|      10:47:00|     8503068|            4|        

In [31]:
%%spark
final_close_trips.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[arrival_time: string, departure_time: string, stop_id: string, stop_sequence: string, pickup_type: string, drop_off_type: string, service_id: string, monday: string, tuesday: string, wednesday: string, thursday: string, friday: string, trip_id: string]

## Create timetable

As request by the paper 

### stops Table

In [32]:
%%spark
stops_table = final_close_trips.select(final_close_trips.stop_id).distinct()
stops_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|    stop_id|
+-----------+
|    8591284|
|    8573729|
|    8503078|
|    8506895|
|    8591190|
|8503306:0:2|
|    8589111|
|    8590819|
|8505004:0:4|
|    8502508|
+-----------+
only showing top 10 rows

In [33]:
%%spark -o all_stops

all_stops = spark.read.csv('/data/sbb/csv/allstops/stop_locations.csv')

all_stops = all_stops.withColumnRenamed('_c0','stop_id')
all_stops = all_stops.withColumnRenamed('_c1','stop_name')
all_stops = all_stops.withColumnRenamed('_c2','stop_lat')
all_stops = all_stops.withColumnRenamed('_c3','stop_lon')
all_stops = all_stops.withColumnRenamed('_c4','location_type')
all_stops = all_stops.withColumnRenamed('_c5','parent_station')

all_stops = all_stops.join(stops_table, "stop_id" ,"inner")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
for i in all_stops["stop_id"]:
    if "Parent" in i:
        print(i)

In [35]:

close_stops_map = all_stops.groupby('stop_id')[['stop_name', 'stop_lat', 'stop_lon', 'parent_station']].apply(lambda g: g.values.tolist()).to_dict()
names_cols = ['stop_name', 'stop_lat', 'stop_lon', 'parent_station']

stops_final = close_stops_map.copy()

for k,v in close_stops_map.items():
    newdict = {}
    for ind, element in enumerate(v[0]):
        newdict[names_cols[ind]] = element
        if names_cols[ind] == "parent_station" and isinstance(element, str):
            if element[:6]=="Parent":
                element = element[6:]
                newdict[names_cols[ind]] = element
            if element[-1]=="P":
                element = element[:-1]
                newdict[names_cols[ind]] = element
            if element != str(int(element)):
                print(element)
    stops_final[k] = newdict



In [36]:
#Line to save the data - commented out as already saved
#pickle.dump(stops_final, open(path_data+"stops.pickle", "wb"))

### connections Table

In [37]:
%%spark
trip_ids_with_days_table = final_close_trips.select('trip_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday').distinct()
trip_ids_with_days_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+---------+--------+------+
|             trip_id|monday|tuesday|wednesday|thursday|friday|
+--------------------+------+-------+---------+--------+------+
|1.TA.26-18-j19-1.1.H|     1|      1|        1|       1|     1|
|1.TA.63-138-Y-j19...|     0|      0|        1|       0|     0|
|1001.TA.26-213-j1...|     0|      0|        0|       0|     1|
|1014.TA.26-520-j1...|     1|      1|        1|       1|     0|
|1017.TA.26-151-j1...|     1|      1|        1|       1|     0|
|1023.TA.26-4-j19-...|     1|      1|        1|       1|     1|
|1025.TA.26-212-j1...|     1|      1|        1|       1|     0|
|1029.TA.26-LAF-j1...|     1|      1|        1|       1|     0|
|103.TA.26-510-j19...|     0|      0|        0|       0|     1|
|1039.TA.26-200-j1...|     0|      0|        0|       0|     1|
+--------------------+------+-------+---------+--------+------+
only showing top 10 rows

In [38]:
%%spark
working_table = final_close_trips.withColumn('stop_sequence',final_close_trips.stop_sequence.cast('int')).orderBy('trip_id', 'stop_sequence').drop('service_id', 'pickup_type', 'drop_off_type', 'monday',  'tuesday', 'thursday', 'wednesday', 'thursday', 'friday')
working_table = working_table.withColumnRenamed('stop_id','departure_id')
working_table = working_table.withColumnRenamed('arrival_time','old_arrival_time')
working_table.show(30)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------+------------+-------------+--------------------+
|old_arrival_time|departure_time|departure_id|stop_sequence|             trip_id|
+----------------+--------------+------------+-------------+--------------------+
|        19:26:00|      19:26:00|     8573720|            1|1.TA.1-205-j19-1.1.R|
|        19:27:00|      19:27:00|     8583206|            2|1.TA.1-205-j19-1.1.R|
|        19:28:00|      19:28:00|     8573725|            3|1.TA.1-205-j19-1.1.R|
|        19:31:00|      19:31:00|     8581666|            4|1.TA.1-205-j19-1.1.R|
|        19:33:00|      19:33:00|     8583805|            5|1.TA.1-205-j19-1.1.R|
|        19:35:00|      19:35:00| 8573726:0:C|            6|1.TA.1-205-j19-1.1.R|
|        15:10:00|      15:10:00|     8573710|            1|1.TA.1-245-j19-1.1.H|
|        15:11:00|      15:11:00|     8591341|            2|1.TA.1-245-j19-1.1.H|
|        15:13:00|      15:13:00|     8502572|            3|1.TA.1-245-j19-1.1.H|
|        15:15:0

In [39]:
%%spark
import pyspark.sql.functions as F
from pyspark.sql.window import Window

w = Window().partitionBy('trip_id').orderBy('stop_sequence')
table_a = working_table.select("*",F.lag("old_arrival_time", -1).over(w).alias("arrival_time"), F.lag("departure_id", -1).over(w).alias("arrival_id")).na.drop()
table_a.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------+------------+-------------+--------------------+------------+-----------+
|old_arrival_time|departure_time|departure_id|stop_sequence|             trip_id|arrival_time| arrival_id|
+----------------+--------------+------------+-------------+--------------------+------------+-----------+
|        10:41:00|      10:41:00|     8503064|            1|1.TA.26-18-j19-1.1.H|    10:45:00|8503065:0:1|
|        10:45:00|      10:45:00| 8503065:0:1|            2|1.TA.26-18-j19-1.1.H|    10:46:00|    8503074|
|        10:46:00|      10:46:00|     8503074|            3|1.TA.26-18-j19-1.1.H|    10:47:00|    8503068|
|        10:47:00|      10:47:00|     8503068|            4|1.TA.26-18-j19-1.1.H|    10:48:00|    8503066|
|        10:48:00|      10:48:00|     8503066|            5|1.TA.26-18-j19-1.1.H|    10:50:00|    8503075|
|        10:50:00|      10:50:00|     8503075|            6|1.TA.26-18-j19-1.1.H|    10:52:00|    8503067|
|        10:52:00|      10:52:00|    

In [40]:
%%spark
import pyspark.sql.functions as F
from pyspark.sql.window import Window

final_table = table_a.join(trip_ids_with_days_table, table_a.trip_id == trip_ids_with_days_table.trip_id)
connections_table = final_table.drop('old_arrival_time', 'stop_sequence').select('departure_id', 'arrival_id', 'departure_time', 'arrival_time', table_a.trip_id, 'monday', 'tuesday', 'wednesday', 'thursday', 'friday')
connections_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------+--------------+------------+--------------------+------+-------+---------+--------+------+
|departure_id| arrival_id|departure_time|arrival_time|             trip_id|monday|tuesday|wednesday|thursday|friday|
+------------+-----------+--------------+------------+--------------------+------+-------+---------+--------+------+
|     8503064|8503065:0:1|      10:41:00|    10:45:00|1.TA.26-18-j19-1.1.H|     1|      1|        1|       1|     1|
| 8503065:0:1|    8503074|      10:45:00|    10:46:00|1.TA.26-18-j19-1.1.H|     1|      1|        1|       1|     1|
|     8503074|    8503068|      10:46:00|    10:47:00|1.TA.26-18-j19-1.1.H|     1|      1|        1|       1|     1|
|     8503068|    8503066|      10:47:00|    10:48:00|1.TA.26-18-j19-1.1.H|     1|      1|        1|       1|     1|
|     8503066|    8503075|      10:48:00|    10:50:00|1.TA.26-18-j19-1.1.H|     1|      1|        1|       1|     1|
|     8503075|    8503067|      10:50:00|    10:52:00|1.TA.26-18

In [41]:
%%spark
connections_table.filter(connections_table.arrival_time < connections_table.departure_time).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [42]:
%%spark
connections_table.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

756285

#### Monday

In [43]:
%%spark -o connections_table_monday_full_sorted
from pyspark.sql.functions import desc
#Monday
#filter for day
connections_table_monday = connections_table.filter(connections_table.monday == '1')
#drop useless columns
connections_table_monday = connections_table_monday.drop('monday', 'tuesday', 'wednesday', 'thursday', 'friday')
#sort 
connections_table_monday_full_sorted = connections_table_monday.orderBy(desc("departure_time"))
#ordering of columns
connections_table_monday_full_sorted = connections_table_monday_full_sorted.select("departure_id", "arrival_id", "departure_time", "arrival_time", "trip_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
#Saving Monday
import pandas as pd
connections_table_monday_full_sorted["departure_time"] = connections_table_monday_full_sorted["departure_time"].apply(lambda x: int(pd.Timestamp(x).timestamp()))

connections_table_monday_full_sorted["arrival_time"] = connections_table_monday_full_sorted["arrival_time"].apply(lambda x: int(pd.Timestamp(x).timestamp()))

result_monday = list(connections_table_monday_full_sorted.itertuples(index=False, name=None))

connections_data= [] #initialize complete list
connections_data.append(result_monday)

#### Tuesday

In [45]:
%%spark -o connections_table_tuesday_full_sorted
#Tuesday
#filter for day
connections_table_tuesday = connections_table.filter(connections_table.tuesday == '1')
#drop useless columns
connections_table_tuesday = connections_table_tuesday.drop('monday', 'tuesday', 'wednesday', 'thursday', 'friday')
#sort 
connections_table_tuesday_full_sorted = connections_table_tuesday.orderBy(desc("departure_time"))
#ordering of columns
connections_table_tuesday_full_sorted = connections_table_tuesday_full_sorted.select("departure_id", "arrival_id", "departure_time", "arrival_time", "trip_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
#Saving Tuesday
connections_table_tuesday_full_sorted["departure_time"] = connections_table_tuesday_full_sorted["departure_time"].apply(lambda x: int(pd.Timestamp(x).timestamp()))

connections_table_tuesday_full_sorted["arrival_time"] = connections_table_tuesday_full_sorted["arrival_time"].apply(lambda x: int(pd.Timestamp(x).timestamp()))

result_tuesday = list(connections_table_tuesday_full_sorted.itertuples(index=False, name=None))

connections_data.append(result_tuesday)

#### Wednesday

In [47]:
%%spark -o connections_table_wednesday_full_sorted
#Wednesday 
#filter for day
connections_table_wednesday = connections_table.filter(connections_table.wednesday == '1')
#drop useless columns
connections_table_wednesday = connections_table_wednesday.drop('monday', 'tuesday', 'wednesday', 'thursday', 'friday')
#sort 
connections_table_wednesday_full_sorted = connections_table_wednesday.orderBy(desc("departure_time"))
#ordering of columns
connections_table_wednesday_full_sorted = connections_table_wednesday_full_sorted.select("departure_id", "arrival_id", "departure_time", "arrival_time", "trip_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
#Saving Wednesday
connections_table_wednesday_full_sorted["departure_time"] = connections_table_wednesday_full_sorted["departure_time"].apply(lambda x: int(pd.Timestamp(x).timestamp()))

connections_table_wednesday_full_sorted["arrival_time"] = connections_table_wednesday_full_sorted["arrival_time"].apply(lambda x: int(pd.Timestamp(x).timestamp()))

result_wednesday = list(connections_table_wednesday_full_sorted.itertuples(index=False, name=None))

connections_data.append(result_wednesday)

#### Thursday

In [49]:
%%spark -o connections_table_thursday_full_sorted
#Thursday
#filter for day
connections_table_thursday = connections_table.filter(connections_table.thursday == '1')
#drop useless columns
connections_table_thursday = connections_table_thursday.drop('monday', 'tuesday', 'wednesday', 'thursday', 'friday')
#sort 
connections_table_thursday_full_sorted = connections_table_thursday.orderBy(desc("departure_time"))
#ordering of columns
connections_table_thursday_full_sorted = connections_table_thursday_full_sorted.select("departure_id", "arrival_id", "departure_time", "arrival_time", "trip_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
#Saving Thursday 
connections_table_thursday_full_sorted["departure_time"] = connections_table_thursday_full_sorted["departure_time"].apply(lambda x: int(pd.Timestamp(x).timestamp()))

connections_table_thursday_full_sorted["arrival_time"] = connections_table_thursday_full_sorted["arrival_time"].apply(lambda x: int(pd.Timestamp(x).timestamp()))

result_thursday = list(connections_table_thursday_full_sorted.itertuples(index=False, name=None))

connections_data.append(result_thursday)

#### Friday

In [51]:
%%spark -o connections_table_friday_full_sorted
#Friday 
#filter for day
connections_table_friday = connections_table.filter(connections_table.friday == '1')
#drop useless columns
connections_table_friday = connections_table_friday.drop('monday', 'tuesday', 'wednesday', 'thursday', 'friday')
#sort 
connections_table_friday_full_sorted = connections_table_friday.orderBy(desc("departure_time"))
#ordering of columns
connections_table_friday_full_sorted = connections_table_friday_full_sorted.select("departure_id", "arrival_id", "departure_time", "arrival_time", "trip_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
#Saving Friday
connections_table_friday_full_sorted["departure_time"] = connections_table_friday_full_sorted["departure_time"].apply(lambda x: int(pd.Timestamp(x).timestamp()))

connections_table_friday_full_sorted["arrival_time"] = connections_table_friday_full_sorted["arrival_time"].apply(lambda x: int(pd.Timestamp(x).timestamp()))

result_friday = list(connections_table_friday_full_sorted.itertuples(index=False, name=None))

connections_data.append(result_friday)

In [53]:
#Saving everything together

pickle.dump(connections_data, open(path_data+"connections_data.pickle", "wb"))


### trips Table

In [54]:
%%spark
final_close_trips.filter(final_close_trips.arrival_time<final_close_trips.departure_time).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

30299

In [55]:
%%spark
final_close_trips.filter(final_close_trips.arrival_time>final_close_trips.departure_time).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [56]:
%%spark
final_close_trips.filter(final_close_trips.arrival_time==final_close_trips.departure_time).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

781267

In [57]:
%%spark
final_close_trips.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

811566

In [58]:
%%spark
#final_close_trips_filtered = final_close_trips.filter(final_close_trips.arrival_time>final_close_trips.departure_time)
trips_table = final_close_trips.select(final_close_trips.trip_id).distinct()
trips_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|             trip_id|
+--------------------+
|1.TA.26-18-j19-1.1.H|
|1.TA.63-138-Y-j19...|
|1001.TA.26-213-j1...|
|1014.TA.26-520-j1...|
|1017.TA.26-151-j1...|
|1023.TA.26-4-j19-...|
|1025.TA.26-212-j1...|
|1029.TA.26-LAF-j1...|
|103.TA.26-510-j19...|
|1039.TA.26-200-j1...|
+--------------------+
only showing top 10 rows

In [59]:
%%spark -o routes_tot_filtered
#trips as described in the readme, but filtered for time and distance

routes_tot_filtered = routes_tot.join(trips_table, "trip_id" ,"inner")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:

routes_tot_filtered_map = routes_tot_filtered.groupby('trip_id')[['route_id', 'service_id', 'trip_headsign', 'trip_short_name', 'direction_id']].apply(lambda g: g.values.tolist()).to_dict()
names_cols = ["route_id", "service_id", "trip_headsign", "trip_short_name", "direction_id"]

map_copied = routes_tot_filtered_map.copy()

for k,v in routes_tot_filtered_map.items():
    newdict = {}
    for ind, element in enumerate(v[0]):
        newdict[names_cols[ind]] = element
    map_copied[k] = newdict


In [61]:
#Line to save the data - commented out as already saved
#pickle.dump(map_copied, open(path_data+"trips.pickle", "wb"))


In [62]:
#!git lfs track "/home/jovyan/work/final-assignment/data/connections_data.pickle"
#!git lfs track "/home/jovyan/work/final-assignment/data/stops.pickle"
#!git lfs track "/home/jovyan/work/final-assignment/data/trips.pickle"

#!git add .gitattributes
#!git commit -m "tutti i vari pickle"
#!git push

### Footpath table

In [68]:
%%spark -o footpath_table

#add attributes lat, lon to stops_table
stops_table_expanded = stops_table.join(all_stops, 'stop_id', 'inner')
#create all possible pairs between stops
stops_table_expanded_a = stops_table_expanded.select([F.col(c).alias("a_"+c) for c in stops_table_expanded.columns])
stops_table_expanded_b = stops_table_expanded.select([F.col(c).alias("b_"+c) for c in stops_table_expanded.columns])
cartesian_product = stops_table_expanded_a.crossJoin(stops_table_expanded_b)
#compute distance for all pairs
stops_table_distance = cartesian_product.withColumn("a", (
        F.pow(F.sin(F.radians(F.col("a_stop_lat") - F.col("b_stop_lat")) / 2), 2) +
        F.cos(F.radians(F.col("b_stop_lat"))) * F.cos(F.radians(F.col("a_stop_lat"))) *
        F.pow(F.sin(F.radians(F.col("a_stop_lon") - F.col("b_stop_lon")) / 2), 2)
    )).withColumn("distance", F.atan2(F.sqrt(F.col("a")), F.sqrt(-F.col("a") + 1)) * 12742000)
#filter pair with distance shorter than 500m
stops_table_distance = stops_table_distance.filter(stops_table_distance.distance <= 500)
#compute time considering walking speed of 50m/1min
#time is in seconds
stops_table_time = stops_table_distance.withColumn('time', F.round((stops_table_distance['distance']/F.lit(50))*F.lit(60), 0).cast('int'))
footpath_table = stops_table_time.select(stops_table_time.a_stop_id.alias('departure_id'), stops_table_time.b_stop_id.alias('arrival_id'), stops_table_time.time)
#remove pairs where arrival == departure
footpath_table = footpath_table.filter(footpath_table.departure_id != footpath_table.arrival_id)
footpath_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------+----+
|departure_id| arrival_id|time|
+------------+-----------+----+
| 8503306:0:2|    8590541| 528|
|     8506895|    8573228| 402|
|     8591284|    8591116| 479|
|     8591190|    8591170| 311|
|     8591284|    8591274| 327|
|     8503078|    8591903| 471|
|     8589111|    8590655| 583|
|     8589111|    8594182| 293|
| 8503306:0:2|8503306:0:1|   0|
|     8589111|    8502208| 351|
+------------+-----------+----+
only showing top 10 rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
#transform list of tuple into dictionary of dictionaries
footpath_map = footpath_table.groupby('arrival_id')[['departure_id', 'time']].apply(lambda g: g.values.tolist()).to_dict()
footpath_map = {k: dict(v) for k, v in footpath_map.items()}
#convert and save the dictionary to pickle
pickle.dump(footpath_map, open(path_data+"footpath_map.pickle", "wb"))

In [70]:
!git lfs track "../data/footpath_map.pickle"
!git add .gitattributes
!git commit -m "footpath_map.pickle"
!git push

Tracking "../data/footpath_map.pickle"
[master 14c70f2] footpath_map.pickle
 2 files changed, 1 insertion(+)
 rewrite data/footpath_map.pickle (98%)
Locking support detected on remote "origin". Consider enabling it with:
  $ git config lfs.https://dslab2022-renku.epfl.ch/gitlab/five-guys/final-assignment.git/info/lfs.locksverify true
Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 48 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 13.67 KiB | 2.73 MiB/s, done.
Total 6 (delta 4), reused 0 (delta 0)
To https://dslab2022-renku.epfl.ch/gitlab/five-guys/final-assignment.git
   da73f68..14c70f2  master -> master


## Standardize stop_id, trip_id for actual_data (istdaten)

#### stop_id

In [71]:
%%spark
actual_data = spark.read.orc('/data/sbb/orc/istdaten')
actual_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- betriebstag: string (nullable = true)
 |-- fahrt_bezeichner: string (nullable = true)
 |-- betreiber_id: string (nullable = true)
 |-- betreiber_abk: string (nullable = true)
 |-- betreiber_name: string (nullable = true)
 |-- produkt_id: string (nullable = true)
 |-- linien_id: string (nullable = true)
 |-- linien_text: string (nullable = true)
 |-- umlauf_id: string (nullable = true)
 |-- verkehrsmittel_text: string (nullable = true)
 |-- zusatzfahrt_tf: string (nullable = true)
 |-- faellt_aus_tf: string (nullable = true)
 |-- bpuic: string (nullable = true)
 |-- haltestellen_name: string (nullable = true)
 |-- ankunftszeit: string (nullable = true)
 |-- an_prognose: string (nullable = true)
 |-- an_prognose_status: string (nullable = true)
 |-- abfahrtszeit: string (nullable = true)
 |-- ab_prognose: string (nullable = true)
 |-- ab_prognose_status: string (nullable = true)
 |-- durchfahrt_tf: string (nullable = true)

In [79]:
%%spark
#original number of stops
stops_table.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2645

In [80]:
%%spark
#remove characters from stop_id in order to have same stops_id as in actual_data
stops_table_clean = stops_table.withColumn('stop_id', F.regexp_replace(F.col("stop_id"), "[:, /, P, Parent]", ""))
stops_table_clean = stops_table.join(all_stops, 'stop_id', 'inner')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
%%spark
#number of stops_table_clean stops contained in actual_data
merged_stops = stops_table_clean.join(actual_data, actual_data.bpuic == stops_table_clean.stop_id, 'inner')
merged_stops.select('stop_id').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1855

In [84]:
%%spark
merged_stops.select(merged_stops.stop_name, merged_stops.haltestellen_name).distinct().show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|           stop_name|   haltestellen_name|
+--------------------+--------------------+
|Zürich, Grünaustr...|Z�rich, Gr�naustr...|
|Unterengstringen,...|Unterengstringen,...|
|Dänikon, Gemeinde...|             Dänikon|
|Zürich, Besenrain...|Z�rich, Besenrain...|
|   Zürich, Beckenhof|   Zürich, Beckenhof|
|Küsnacht ZH, Ob. ...|K�snacht ZH, Ober...|
|Zürich, Einfangst...|Zürich, Einfangst...|
|Wermatswil, Chamm...|Wermatswil, Chamm...|
|Herrliberg, Harze...|Herrliberg, Harze...|
|    Zürich, Maienweg|    Zürich, Maienweg|
+--------------------+--------------------+
only showing top 10 rows

In [92]:
%%spark
stops_table_clean.select('stop_id').show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


In [89]:
%%spark
#study excluded stops
excluded_stops = stops_table_clean.select('stop_id').subtract(merged_stops.select('stop_id')).distinct()
#add attributes to excluded stops
# excluded_stops = all_stops.join(excluded_stops, 'stop_id')
excluded_stops.show(10, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o2121.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 25 in stage 802.0 failed 4 times, most recent failure: Lost task 25.3 in stage 802.0 (TID 44043, iccluster057.iccluster.epfl.ch, executor 5): java.io.FileNotFoundException: /hdata/sdg/hadoop/yarn/local/usercache/eric/appcache/application_1652960972356_1257/blockmgr-a22a75f2-45a0-412e-bee1-3fcbaca03e30/21/temp_shuffle_3ed82d97-a8ec-44f3-9257-7d411f853b20 (Input/output error)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuf

In [88]:
%%spark
#check if some excluded stops have different stop_id in actual_data, but same  stop_name
actual_data.join(excluded_stops, actual_data.haltestellen_name == excluded_stops.stop_name, 'inner').select('bpuic', 'stop_id', 'stop_name', 'haltestellen_name').distinct().show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o2070.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 22 in stage 774.0 failed 4 times, most recent failure: Lost task 22.3 in stage 774.0 (TID 43191, iccluster057.iccluster.epfl.ch, executor 5): java.io.FileNotFoundException: /hdata/sdg/hadoop/yarn/local/usercache/eric/appcache/application_1652960972356_1257/blockmgr-a22a75f2-45a0-412e-bee1-3fcbaca03e30/22/temp_shuffle_f15a7539-f005-40e7-86b4-0f90c6da6d6d (Input/output error)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuf

#### trip_id

In [93]:
%%spark

final_close_trips.show(10, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+-----------+-------------+-----------+-------------+----------+------+-------+---------+--------+------+--------------------+
|arrival_time|departure_time|stop_id    |stop_sequence|pickup_type|drop_off_type|service_id|monday|tuesday|wednesday|thursday|friday|trip_id             |
+------------+--------------+-----------+-------------+-----------+-------------+----------+------+-------+---------+--------+------+--------------------+
|10:41:00    |10:41:00      |8503064    |1            |0          |0            |TA+b0b46  |1     |1      |1        |1       |1     |1.TA.26-18-j19-1.1.H|
|10:45:00    |10:45:00      |8503065:0:1|2            |0          |0            |TA+b0b46  |1     |1      |1        |1       |1     |1.TA.26-18-j19-1.1.H|
|10:46:00    |10:46:00      |8503074    |3            |0          |0            |TA+b0b46  |1     |1      |1        |1       |1     |1.TA.26-18-j19-1.1.H|
|10:47:00    |10:47:00      |8503068    |4            |0          |0  

In [94]:
1.TA.26-18-j19-1.1.H

SyntaxError: invalid syntax (<ipython-input-94-41ed6173d310>, line 1)

In [97]:
%%spark
routes = spark.read.csv('/data/sbb/csv/routes/2019/05/08/routes.txt', header=True)
routes.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+----------------+---------------+----------+----------+
|   route_id|agency_id|route_short_name|route_long_name|route_desc|route_type|
+-----------+---------+----------------+---------------+----------+----------+
|11-40-j19-1|      801|             040|           null|       Bus|       700|
|11-61-j19-1|     7031|             061|           null|       Bus|       700|
|11-62-j19-1|     7031|             062|           null|       Bus|       700|
|24-64-j19-1|      801|             064|           null|       Bus|       700|
|11-83-j19-1|      801|             083|           null|       Bus|       700|
|1-1-A-j19-1|       11|               1|           null|    S-Bahn|       400|
|1-1-B-j19-1|       11|               1|           null|    S-Bahn|       400|
|1-1-C-j19-1|      723|               1|           null|       Bus|       700|
|1-1-D-j19-1|      840|               1|           null|       Bus|       700|
|1-1-E-j19-1|      886|               1|           n

In [ ]:
%%spark

actual_data.show(2, truncate=False)

In [ ]:
%%spark

actual_data.select('linien_text').distinct().show(10, truncate=False)